In [91]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Imports

In [107]:
import torch
import pyterrier as pt 
from transformers import BertTokenizer
from model.Bert import Bert
from model.Train import Trainer
from model.Loss import LSmoothing

In [109]:
if not pt.started():
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

Exception: Unable to find JAVA_HOME

## Variables

In [102]:
nb_labels = 2
epochs = 100
learning_rate = 1e-4
device = "cuda" if torch.cuda.is_available() else "cpu"

## Loading Data

In [106]:
def isIn(string):
    return "trec" in string

all_datasets = pt.datasets.list_datasets()
all_datasets

AttributeError: module 'pyterrier' has no attribute 'datasets'

In [105]:
dataset = pt.datasets.get_dataset()

AttributeError: module 'pyterrier' has no attribute 'datasets'

## Training

In [ ]:
model_name = "bert-base-uncased"
model = Bert(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)
optimizer = torch.optim.AdamW
loss = LSmoothing(nb_labels=2)
trainer = Trainer()
history = trainer.set_model(model)\
    .set_loader(train_loader, val_loader, test_loader)\
    .set_loss_fn(loss)\
    .set_optimizer(optimizer)\
    .fit(learning_rate, epochs)

In [98]:
def predict_sentiment(text, model, tokenizer, device, max_length=128):
    model.eval()
    encoding = tokenizer(text, return_tensors='pt', max_length=max_length, padding='max_length', truncation=True)
    inputs = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    with torch.no_grad():
        outputs = model(inputs=inputs, attention_mask=attention_mask)
        print(outputs)
        print(1-outputs)
        #loss = loss_fn(outputs, [1,0])
        #print(loss)
        #print(loss[0])
        _, preds = torch.max(outputs, dim=1)

    return "positive" if preds.item() == 1 else "negative"

In [99]:
test_text = "The movie was great and I really enjoyed the performances of the actors."
sentiment = predict_sentiment(test_text, model, tokenizer, device)
print("The movie was great and I really enjoyed the performances of the actors.")
print(f"Predicted sentiment: {sentiment}")

tensor([[-0.1116, -0.1323]], device='cuda:0')
tensor([[1.1116, 1.1323]], device='cuda:0')
The movie was great and I really enjoyed the performances of the actors.
Predicted sentiment: negative
